In [1]:
import numpy as np
import time
import pickle
import operator
from numpy.random import uniform
import matplotlib.pyplot as plt

In [2]:
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None, action=None):
        self.parent = parent
        self.position = position
        self.action = action

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position


def get_closest_grid(current_pos):
    differences = current_pos-uncertainty_grids
    distances = np.sum(differences*differences,axis=1)
    min_ind = np.argmin(distances)
#     print ("read pos: {0} index: {1} found_pos: {2}".format(current_pos, min_ind, uncertainty_grids[min_ind]))
    
    return min_ind

def get_obstacle_indices():
    grid_res = 1.0
    obstacle_indices = []
    obstacle_indices_unsquezed = []
    
    obstacle_start = np.array([[-20, -20], [-20, -20], [19, -20], [-20, 19]]) 
    obstacle_end = np.array([[-19, 20], [20, -19], [20, 20], [20, 20]])

    for i in range(obstacle_start.shape[0]):
        x_range = np.arange(-grid_res/2+obstacle_start[i,0], obstacle_end[i,0]+grid_res/2, grid_res/4)
        y_range = np.arange(-grid_res/2+obstacle_start[i,1], obstacle_end[i,1]+grid_res/2, grid_res/4)
        indices = []
        for x in x_range:
            for y in y_range:
                current_pos = np.array([x,y])
                current_ind = get_closest_grid(current_pos)
                if current_ind not in indices:
                    indices.append(current_ind)

        obstacle_indices.append(indices)
        
    for i in range(len(obstacle_indices)):
        for j in range(len(obstacle_indices[0])):
            obstacle_indices_unsquezed.append(obstacle_indices[i][j])

    return obstacle_indices_unsquezed

In [6]:
#Grid information
grid_res = 2.0
x_lim, y_lim, z_lim = 20, 20, 6
obstacle_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 42, 63, 84, 105, 
                    126, 147, 168, 189, 210, 231, 252, 273, 294, 315, 336, 357, 378, 399, 420, 421, 422, 423, 424, 
                    425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 41, 62, 83, 
                    104, 125, 146, 167, 188, 209, 230, 251, 272, 293, 314, 335, 356, 377, 398, 419, 190, 191, 192, 
                    193, 194, 195, 196, 197, 211, 212, 213, 214, 215, 216, 217, 218, 200, 201, 202, 203, 204, 205, 
                    206, 207, 208, 221, 222, 223, 224, 225, 226, 227, 228, 229]

def astar_drone(start, end, obstacle_indices=None):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""
    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0
    visited_grids = []

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:
        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append([current.position, current.action])
                visited_grids.append(get_closest_grid(current.position))
                current = current.parent
            return path[::-1], visited_grids # Return reversed path and visited grids

        # Generate children
        children = []
        for index, new_position in enumerate([(0, -grid_res), (0, grid_res), (-grid_res, 0), (grid_res, 0), 
                                              (grid_res, grid_res), (grid_res, -grid_res), 
                                              (-grid_res, grid_res), (-grid_res, -grid_res)]): # Adjacent squares
#         for new_position in [(0, -grid_res, 0), (0, grid_res, 0), (-grid_res, 0, 0), (grid_res, 0, 0)]: # Adjacent squares
            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
#             print ("Node position: ", node_position)
            node_index = get_closest_grid(node_position)
#             print ("Node index: {0} Node pos: {1}/{2}".format(node_index, uncertainty_grids[node_index], node_position))
            
            
            # Make sure within range
            if node_position[0] > x_lim or node_position[0] < -x_lim or node_position[1] > y_lim or node_position[1] < -y_lim:
#                 print ("It's not within the range. Node position: ", node_position)
                continue
              
                
            if node_index in obstacle_indices:
                print ("It's a obstacle place. Node position: ", node_position)
                continue
                    
                

            # Create new node
            new_node = Node(current_node, node_position, index)
            
            # Append
            children.append(new_node)

        # Loop through children
        for child in children:
            
            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue
            # Add the child to the open list
            open_list.append(child)

In [12]:
import time 

x_lim, y_lim, z_lim = 20, 20, 6
eps = 0.1
res = 2
X,Y = np.mgrid[-x_lim : x_lim + eps:res, 
               -y_lim : y_lim + eps:res]
uncertainty_grids = np.vstack((X.flatten(), Y.flatten())).T
    
t = time.time()

start = (-18., -18.)
end = (12.,  10.)

# end = (4., 14.)
end = (12., 16.)
end = (-16., -12.)
end = (-12.,  16.)
end = (8., 8.)
end = (-12.,  12.)
    
    
path = astar_drone(start, end, obstacle_indices=obstacle_indices)
print(path)
elapsed = time.time() - t

It's a obstacle place. Node position:  (-18.0, -20.0)
It's a obstacle place. Node position:  (-20.0, -18.0)
It's a obstacle place. Node position:  (-16.0, -20.0)
It's a obstacle place. Node position:  (-20.0, -16.0)
It's a obstacle place. Node position:  (-20.0, -20.0)
([[(-18.0, -18.0), None], [(-16.0, -16.0), 4], [(-14.0, -14.0), 4], [(-12.0, -12.0), 4], [(-12.0, -10.0), 1], [(-12.0, -8.0), 1], [(-12.0, -6.0), 1], [(-12.0, -4.0), 1], [(-12.0, -2.0), 1], [(-12.0, 0.0), 1], [(-12.0, 2.0), 1], [(-12.0, 4.0), 1], [(-12.0, 6.0), 1], [(-12.0, 8.0), 1], [(-12.0, 10.0), 1], [(-12.0, 12.0), 1]], [100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 66, 44, 22])


In [13]:
np.arange(1,5)

array([1, 2, 3, 4])

In [12]:
uncertainty_grids[obstacle_indices]

array([[-20., -20.],
       [-20., -18.],
       [-20., -16.],
       [-20., -14.],
       [-20., -12.],
       [-20., -10.],
       [-20.,  -8.],
       [-20.,  -6.],
       [-20.,  -4.],
       [-20.,  -2.],
       [-20.,   0.],
       [-20.,   2.],
       [-20.,   4.],
       [-20.,   6.],
       [-20.,   8.],
       [-20.,  10.],
       [-20.,  12.],
       [-20.,  14.],
       [-20.,  16.],
       [-20.,  18.],
       [-20.,  20.],
       [-18., -20.],
       [-16., -20.],
       [-14., -20.],
       [-12., -20.],
       [-10., -20.],
       [ -8., -20.],
       [ -6., -20.],
       [ -4., -20.],
       [ -2., -20.],
       [  0., -20.],
       [  2., -20.],
       [  4., -20.],
       [  6., -20.],
       [  8., -20.],
       [ 10., -20.],
       [ 12., -20.],
       [ 14., -20.],
       [ 16., -20.],
       [ 18., -20.],
       [ 20., -20.],
       [ 20., -18.],
       [ 20., -16.],
       [ 20., -14.],
       [ 20., -12.],
       [ 20., -10.],
       [ 20.,  -8.],
       [ 20.,

In [92]:
init_range = np.arange(-18, 0, 2)

In [93]:
np.random.choice(init_range, (5,), replace=False)

array([ -2, -16, -10, -18,  -4])

In [125]:
map_lim = 20.0 - 2.0
init_pos_list = []
init_grid_list = []
for i in range(5):
    init_grid_list.append(get_closest_grid([-map_lim + 2*i, -map_lim]))
    init_pos_list.append([-map_lim + 2*i, -map_lim])
for i in range(1,6):
    init_grid_list.append(get_closest_grid([-map_lim , -map_lim + 2*i]))
    init_pos_list.append([-map_lim , -map_lim + 2*i])

## Reward List

In [148]:
with open('model_reward_list.pkl', 'rb') as f:  
    model_reward_dict = pickle.load(f)
model_reward_list = sorted(model_reward_dict.items(), key=operator.itemgetter(1))
model_reward_list[-1]

In [19]:
np.random.seed(1)
for i in range(5):
#     print (np.random.rand(5))
    print (np.random.choice(np.arange(5)))

3
4
0
1
3


In [20]:
np.random.seed(1)
for i in range(5):
    print (np.random.choice(np.arange(5)))

3
4
0
1
3


In [14]:
a = 149
a_list = np.array([i for i in range(150)])
np.any(a_list == a)

True

True

In [9]:
a_list

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149])